# Gazete Tiraj Verileri
Bu script, haftalık gazete satış rakamlarını çekmektedir. Aşağıda verilen kaynakta haftalık olarak en çok satana göre 20 gazete sıralanmaktadır.

**Senaryo**: Sayfada veri ajax ile güncellenmektedir. Sitenin sağ üst köşesinde yer alan tarih seçici ile hafta değiştirilir. Yıl içerisindeki her pazartesi seçilerek sayfada bulunan tablo güncellenir. Güncellemeden sonra sayfadaki tablo çekilerek DataFrame'e dönüştürülür ve bir listeye eklenir. Bir sonraki Pazartesi'ye geçilir.

**Kapsam**: Görünüşe göre web sitesinde 2015'e kadar veri bulunmakta. 2015-2019 aralığındaki veriler yıllık olarak çekilmiştir.

**Başlıklar**

*   date (Tarih)
*   newspaper (Gazete)
*   circulation (Tiraj)

**Erişim tarihi**: 01.11.2019

**Kaynak**: gazetetirajları.com

**Not**: Gazete tirajları resmi bir kaynaktan yapısal bir şekilde paylaşılmadığından, en sağlıklı kaynak olarak bu web sitesi seçilmiştir. Barındırdığı satış rakamlarının güvenilirliği test edilememektedir.

Umut Irmaksever @ AVVG

Gerekli paketler yüklenir ve Selenium instance'ı başlatılır, web sayfasına gidilir.

In [0]:
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!pip install selenium html5lib bs4
!pip install BeautifulSoup4
!pip install html5lib
!pip install matplotlib
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By as By

import html5lib
import pandas as pd

import datetime
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1080")
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
page = wd.get("http://gazetetirajlari.com/HaftalikTirajlar.aspx")

Bir yıl içerisindeki tüm pazartesileri bir listeye kaydedelim.

In [0]:
def allmondays(year):
    # Yılın ilk günü
    d = datetime.date(year, 1, 1)
    # Pazartesi'yi bul
    d += datetime.timedelta(days=7-d.weekday())
    # Yıl aynı olduğu sürece 7 gün artırarak devam et 
    while d.year == year:
        yield d
        d+= datetime.timedelta(days=7)
    return(d)
# Listedeki tüm tarih objelerini istenen formata sok-
def get_mondays_list(year):
    mondays = allmondays(year)
    mondays = list(map(lambda x: x.strftime("%d.%m.%Y"), mondays))
    return mondays


Sayfadaki veri tablosunu bulup DataFrame'e çevirelim.

In [0]:
def get_circulation_data():
    data_table_div = wd.find_element_by_id("divHaftalikTirajAra")
    week = data_table_div.find_elements_by_tag_name("h3")[0].text
    data_table = data_table_div.find_elements_by_tag_name("table")[0].get_attribute("outerHTML")
    data_table_pd = pd.read_html(data_table, flavor="bs4")
    data_table_pd = data_table_pd[0]
    return data_table_pd

Sayfadaki veri tablosunun üstünde başlık olarak o haftanın tarih aralığı belirtiliyor. Doğrulama amaçlı kullanılabilir. İkinci fonksiyon ise datepicker widget'ına istenen tarihi girip butona basıyor.

In [0]:
def get_current_week():
    week = wd.find_element_by_id("divHaftalikTirajAra").find_elements_by_tag_name("h3")[0].text
    return week
def change_date(date):
    date_textfield = wd.find_element_by_id("txtHaftalikTirajTarih")
    date_textfield.clear()
    # date_textfield.click()
    date_textfield.send_keys(date)
    wd.find_element_by_id("btnHaftalikTirajAra").click()
    # Tarih değiştirildikten sonra bir "Yükleniyor" mesajıyla bir modal çıkıyor.
    # Scriptin sekteye uğramaması için bu modal kapanana kadar bekle.
    element = WebDriverWait(wd, 10).until(
        EC.invisibility_of_element_located((By.ID, 'loading')))

Aşağıdaki helper (uardımcı) fonksiyon ile bir ya da daha fazla yıl içerisindeki Pazartesiler datepicker'da deneniyor ve o hafta için veri olup olmadığı belirleniyor. Veri bulunan haftalar "success_weeks" listesine, olmayan haftalar ise "fail_weeks" listesine kaydediliyor.

In [0]:
weeks_dict = {}
years = range(2011, 2012)
success_weeks = []
fail_weeks = []
def check_if_week_exists(years):
    for year in years:
        mondays_list = get_mondays_list(year)
        for week in mondays_list:
            change_date(week)
            try:
                week_from_website = get_current_week()
                print(week, week_from_website)
                # Hem python tarafından sağlanan tarih, hem de web sitesinden
                # alınan hafta stringini kaydet.
                success_weeks.append([week, week_from_website])
            except IndexError:
                print("nonexistent")
                fail_weeks.append([week, "fail"])
check_if_week_exists(years)

Veri olmayan haftaları kaydedelim.

In [0]:
# Fail olan haftaları bir dosyaya yazmak için aşağıdaki dosya ismini değiştirebilirsiniz.
with open("fails_2015_2.txt", mode="w+")as file:
    for fail in fail_weeks:
        file.write(str(fail) + "\n")
    file.close()

Her hafta için veri tablosunu bir dict objesine kaydedelim.

In [0]:
weekly_circulation_list = {}
for week in success_weeks:
    print(week)
    change_date(week[0])
    sleep(0.5)
    circulation_data = get_circulation_data()
    # print(circulation_data)
    weekly_circulation_list[week[0]] = circulation_data

Bir yıldaki tüm veri olan haftaların bulunduğu dict objesindeki değerler DataFrame şeklinde. Web sayfasındaki tabloyu bire bir kaydettiğimiz için sadece ihtiyacımız olanları alalım ve DataFrame listesini düzleştirelim.

In [0]:
rows_list = []
for circulation_table in weekly_circulation_list.keys():
    week = circulation_table
    table = weekly_circulation_list[circulation_table]
    for row in table.index:
        newspaper_name = table["GAZETE ADI"][row]
        sales = table["GÜNCEL"][row]
        rows_list.append([week, newspaper_name, sales])
        print(week, newspaper_name, sales)
    print(circulation_table)

Düzleştirilmiş listeyi tekrardan DataFrame'e çevirerek csv formatında kaydedelim.

In [0]:
rows_list_pd = pd.DataFrame(rows_list, columns=["date", "newspaper", "circulation"])
rows_list_pd.to_csv("tr_newspapers_2015.csv")
rows_list_pd-